In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

In [2]:
data = pd.read_csv(r"filtered_gpt4.csv")
# data = data.rename(columns={'Исполнитель':'executor', 'Группа тем': 'topic_group', 'Текст инцидента':'text', 'Тема': 'topic'})

In [3]:
data['label'] = data['label'].apply(lambda x: x.split(')')[0])

In [4]:
import re

def clean_text(text):
    # Удаление ссылок
    text = re.sub(r'http\S+|www.\S+', '', text)
    
    # Удаление смайликов
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)
    
    # Удаление всех символов, кроме букв, цифр и знаков пунктуации
    text = re.sub(r'[^\w\s\.\,\!\?]', '', text)
    
    return text

data['text'] = data['text'].apply(clean_text)

In [6]:
data.head(5)

,Unnamed: 0,text,label
0,0,"Блиновская приехала в суд \n\nСегодня решается вопрос о продлении домашнего ареста. На заседании королева марафонов назвала себя пророссийским человеком и расплакалась. Также Елена пожаловалась, что электронный браслет ей давит и ограничивает её желание посещать церковь и играть с детьми на площадке. \n\nАдвокат Елены рассказал, что она выплатила 30 млн из 918 млн налогов, в неуплате которых ее обвиняют.\n\nВся в белом",15
1,1,Новое руководство FTX смогло вернуть около 7 миллиардов в ликвидных активах.\n\nНативный токен биржи FTT отреагировал ростом на 20.\n\nDeCenter,8
2,2,Владелец коллекции NFT потерял более 570 000 изза мошенников \n \nПользователь под ником s27 лишился невзаимозаменяемых токенов BAYC и NFT из коллекции Mutant Ape под номерами и Владельца токенов обманули при обмене. \n \n \n \n\nИсточник,8
3,3,"Штaт HьюЙopк paccмoтpит пpинятиe кpиптoвaлют гocучpeждeниями\n\nЗaкoнoпpoeкт уже внecли в Accaмблeю штaтa HьюЙopк. Если одобрят, то гocyдapcтвeнные учpeждeния начнут пpинимaть кpипту для oплaты штpaфoв, гpaждaнcкoпpaвoвыx взыcкaний, нaлoгoв, cбopoв и любых дpугиx плaтeжeй, которые взыскивает гocудapcтв\n\nПравда, для этого законопроект должны принять как в Accaмблee, так и в Ceнaте штaтa HьюЙopк, и обязательно должен подписать губepнaтop штaтa Kэти Xoкул",8
4,4,"Tkillah подарил цепь с бриллиантами Bustarhymes в честь 50летия хипхопа.\n\nПоказал гостеприимство и кинул респекта от русских, получается.",29


In [7]:
category_counts = data['label'].value_counts()
print(category_counts)

label
3     1432
5      958
8      900
13     849
6      801
16     779
4      769
29     560
1      471
9      430
18     321
22     229
7      172
19     160
23     103
10      98
17      79
20      74
28      49
24      46
11      34
25      32
14      32
2       27
15      26
12      26
21      14
26      13
27       6
Name: count, dtype: int64


In [8]:
data.label.nunique()

29

In [11]:
Х = data['text']
y = data['label']

X_train, X_rem, y_train, y_rem = train_test_split(Х ,y, train_size=0.7, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, stratify=y_rem)

train_data = pd.concat([X_train, y_train], axis=1)
valid_data = pd.concat([X_valid, y_valid], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_data.columns =['text', 'label']
valid_data.columns =['text', 'label']
test_data.columns =['text', 'label']

train_data.to_csv(r"train_data.csv", sep='\t')
valid_data.to_csv(r"valid_data.csv", sep='\t')
test_data.to_csv(r"test_data.csv", sep='\t')

In [7]:
from bert_classifier import BertClassifier

c:\Users\Chubu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
classifier = BertClassifier(model_path=r'C:\Users\Chubu\OneDrive\Рабочий стол\last_CP\NLP\NaturaLP_LaBSE_topic_group.pt',
                            tokenizer_path='cointegrated/LaBSE-en-ru')

In [9]:
text = 'Здравствуйте, скажите пожалуйста сколько времени занимает подпись документов на операцию в г Лысьва?В понедельник были переданы документы на подпись на операцию в г Перми сегодня четверг документы до сих пор не пришли. Операция была назначена на 13 сентября.Из Москвы посылка пришла быстрее.'

In [10]:
topic_group = classifier.predict(text)

In [13]:
collector = dict(enumerate(list(data.topic_group.unique())))

In [15]:
collector

{0: 'Благоустройство',
 1: 'Социальное обслуживание и защита',
 2: 'Общественный транспорт',
 3: 'Здравоохранение/Медицина',
 4: 'Мусор/Свалки/ТКО',
 5: 'Образование',
 6: 'Дороги',
 7: 'ЖКХ',
 8: 'Коронавирус',
 9: 'Экономика и бизнес',
 10: 'Культура',
 11: 'Связь и телевидение',
 12: 'Газ и топливо',
 13: 'Безопасность',
 14: 'Спецпроекты',
 15: 'Мобилизация',
 16: 'МФЦ "Мои документы"',
 17: 'Физическая культура и спорт',
 18: 'Торговля',
 19: 'Строительство и архитектура',
 20: 'Памятники и объекты культурного наследия',
 21: 'Экология',
 22: 'Государственная собственность',
 23: 'Роспотребнадзор',
 24: 'Погребение и похоронное дело',
 25: 'Электроснабжение'}

In [14]:
collector[topic_group]

'Здравоохранение/Медицина'

In [12]:
train_data = pd.read_csv('train_data.csv', encoding='utf-8', delimiter='\t')
valid_data = pd.read_csv('valid_data.csv', encoding='utf-8', delimiter='\t')
test_data = pd.read_csv('test_data.csv', encoding='utf-8', delimiter='\t')

In [13]:
print(train_data.label.nunique())
print(valid_data.label.nunique())
print(test_data.label.nunique())

29
29
29
